##### **Install Package and Libraries**

In [ ]:
# Install package
!pip install selenium
!pip install webdriver-manager

# Install library
import datetime as dt
import datetime
import numpy as np
import os
import pandas as pd
from selenium import webdriver

# Install wget
!apt-get install wget

# Download the chromedriver.exe from GitHub
os.system('wget https://github.com/SamapanThongmee/Scrap_SET50_Index_Options_Data/blob/main/chromedriver.exe')

def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver

driver = web_driver()

##### **Check Options Symbol Year-by-Year**

In [24]:
def getCheckOptions(symbol='S50Z23P900'):
    options = []
    try:
        options_df = pd.DataFrame()

        for page in range(1):
            url = f'https://www.tfex.co.th/tfex/historicalTrading.html?symbol={symbol}&series=&page={page}&periodView=A&locale=en_US'
            driver.get(url)
            data = driver.page_source
            scrap = pd.read_html(data)[0]

            mask = scrap['Date'] == 'Grand Total'
            indices = int(scrap.index[mask][0])
            options_data = scrap.iloc[:indices, :][['Date', 'Open', 'High', 'Low', 'Close', 'SP', 'Vol', 'OI']].replace('-', 0)

            options_data['Date'] = pd.to_datetime(options_data['Date'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
            options_data['SP'] = pd.to_numeric(options_data['SP'])
            options_data['Vol'] = pd.to_numeric(options_data['Vol'])
            options_data['OI'] = pd.to_numeric(options_data['OI'])

            options_df = pd.concat([options_df, options_data], axis=0)

        options_df['Symbol'] = symbol
        options_df = options_df[['Date', 'Symbol', 'Open', 'High', 'Low', 'Close', 'SP', 'Vol', 'OI']]
        options_df = options_df.sort_values(by='Date').reset_index(drop=True)
        options_df = options_df.drop_duplicates(subset=['Date'], keep='last')
        options = options_df

    except:
      pass

    return options

In [25]:
def generate_option_symbols(year, letters):
    symbols_to_check = {}

    for letter in letters:
        current_list_C = []

        start_value = 250
        end_value = 1250
        interval = 25

        current_value = start_value
        while current_value <= end_value:
            symbol_C = f'S50{letter}{year}C{current_value}'
            current_list_C.append(symbol_C)
            current_value += interval

        symbols_to_check[f'S50{letter}{year}_C'] = current_list_C

    symbols_to_check = {key.replace('_C', '_Call'): value for key, value in symbols_to_check.items()}

    all_symbols = []
    for value in symbols_to_check.values():
        all_symbols.extend(value)

    series_options = []

    for symbol in all_symbols:
        options = getCheckOptions(symbol)

        if len(options) > 0:
            series_options.append(symbol)

    Call_Options_Symbols = series_options
    Put_Options_Symbols = [symbol.replace('C', 'P') for symbol in Call_Options_Symbols]
    return Call_Options_Symbols + Put_Options_Symbols

current_year = datetime.datetime.now().year
last_two_digits = current_year % 100
letters = ['M', 'H', 'U', 'Z']

for year in range(7, last_two_digits + 1):
    year_str = str(year).zfill(2)
    symbols = generate_option_symbols(year_str, letters)

    with open(f'{year + 2000}_Options_Symbols.txt', 'w') as file:
      file.write(' '.join(map(str, symbols)))

##### **Pull Options Data**

In [26]:
def getOptions(symbol='S50U23P900'):
    options = []
    try:
        options_df = pd.DataFrame()

        for page in range(15):
            url = f'https://www.tfex.co.th/tfex/historicalTrading.html?symbol={symbol}&series=&page={page}&periodView=A&locale=en_US'
            driver.get(url)
            data = driver.page_source
            scrap = pd.read_html(data)[0]

            mask = scrap['Date'] == 'Grand Total'
            indices = int(scrap.index[mask][0])
            options_data = scrap.iloc[:indices, :][['Date', 'Open', 'High', 'Low', 'Close', 'SP', 'Vol', 'OI']].replace('-', 0)

            options_data['Date'] = pd.to_datetime(options_data['Date'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
            options_data['SP'] = pd.to_numeric(options_data['SP'])
            options_data['Vol'] = pd.to_numeric(options_data['Vol'])
            options_data['OI'] = pd.to_numeric(options_data['OI'])

            options_df = pd.concat([options_df, options_data], axis=0)

        options_df['Symbol'] = symbol
        options_df = options_df[['Date', 'Symbol', 'Open', 'High', 'Low', 'Close', 'SP', 'Vol', 'OI']]
        options_df = options_df.sort_values(by='Date').reset_index(drop=True)
        options_df = options_df.drop_duplicates(subset=['Date'], keep='last')
        options = options_df

    except Exception as e:
        print(f"An error occurred: {e}")

    return options

def get_options_data(options_list):
  options_data = []
  for symbol in options_list:
    data = getOptions(symbol=symbol)
    options_data.append(data)
  options_data = pd.concat(options_data, ignore_index=True)

##### **Manipulate Options Series**

In [ ]:
options_symbols = {}
for year in range(2007, 2024):
    file_name = f'{year}_Options_Symbols.txt'
    with open(file_name, 'r') as file:
        text = file.read()
        options_symbols[year] = text.split()

optionsSymbol_2007 = ['S50Z07C550', 'S50Z07C600', 'S50Z07C650', 'S50Z07C700', 'S50Z07P550', 'S50Z07P600', 'S50Z07P650', 'S50Z07P700']
optionsSymbol_2008 = ['S50M08C500', 'S50M08C550', 'S50M08C600', 'S50M08C650', 'S50M08C700', 'S50H08C500', 'S50H08C550', 'S50H08C600', 'S50H08C650', 'S50H08C700', 'S50U08C400', 'S50U08C450', 'S50U08C500', 'S50U08C550', 'S50U08C600', 'S50U08C650', 'S50U08C700', 'S50Z08C250', 'S50Z08C300', 'S50Z08C350', 'S50Z08C400', 'S50Z08C450', 'S50Z08C500', 'S50Z08C550', 'S50Z08C600', 'S50Z08C650', 'S50M08P500', 'S50M08P550', 'S50M08P600', 'S50M08P650', 'S50M08P700', 'S50H08P500', 'S50H08P550', 'S50H08P600', 'S50H08P650', 'S50H08P700', 'S50U08P400', 'S50U08P450', 'S50U08P500', 'S50U08P550', 'S50U08P600', 'S50U08P650', 'S50U08P700', 'S50Z08P250', 'S50Z08P300', 'S50Z08P350', 'S50Z08P400', 'S50Z08P450', 'S50Z08P500', 'S50Z08P550', 'S50Z08P600', 'S50Z08P650']
optionsSymbol_2009 = ['S50M09C250', 'S50M09C300', 'S50M09C350', 'S50M09C400', 'S50M09C450', 'S50M09C500', 'S50M09C550', 'S50M09C600', 'S50H09C250', 'S50H09C300', 'S50H09C350', 'S50H09C400', 'S50H09C450', 'S50H09C500', 'S50H09C550', 'S50H09C600', 'S50H09C650', 'S50U09C250', 'S50U09C300', 'S50U09C350', 'S50U09C400', 'S50U09C450', 'S50U09C500', 'S50U09C550', 'S50Z09C250', 'S50Z09C300', 'S50Z09C350', 'S50Z09C400', 'S50Z09C450', 'S50Z09C500', 'S50Z09C550', 'S50M09P250', 'S50M09P300', 'S50M09P350', 'S50M09P400', 'S50M09P450', 'S50M09P500', 'S50M09P550', 'S50M09P600', 'S50H09P250', 'S50H09P300', 'S50H09P350', 'S50H09P400', 'S50H09P450', 'S50H09P500', 'S50H09P550', 'S50H09P600', 'S50H09P650', 'S50U09P250', 'S50U09P300', 'S50U09P350', 'S50U09P400', 'S50U09P450', 'S50U09P500', 'S50U09P550', 'S50Z09P250', 'S50Z09P300', 'S50Z09P350', 'S50Z09P400', 'S50Z09P450', 'S50Z09P500', 'S50Z09P550']
optionsSymbol_2010 = ['S50M10C350', 'S50M10C400', 'S50M10C450', 'S50M10C500', 'S50M10C550', 'S50M10C600', 'S50H10C250', 'S50H10C300', 'S50H10C350', 'S50H10C400', 'S50H10C450', 'S50H10C500', 'S50H10C550', 'S50H10C600', 'S50U10C450', 'S50U10C500', 'S50U10C550', 'S50U10C600', 'S50U10C650', 'S50U10C700', 'S50Z10C450', 'S50Z10C500', 'S50Z10C550', 'S50Z10C600', 'S50Z10C650', 'S50Z10C700', 'S50Z10C750', 'S50M10P350', 'S50M10P400', 'S50M10P450', 'S50M10P500', 'S50M10P550', 'S50M10P600', 'S50H10P250', 'S50H10P300', 'S50H10P350', 'S50H10P400', 'S50H10P450', 'S50H10P500', 'S50H10P550', 'S50H10P600', 'S50U10P450', 'S50U10P500', 'S50U10P550', 'S50U10P600', 'S50U10P650', 'S50U10P700', 'S50Z10P450', 'S50Z10P500', 'S50Z10P550', 'S50Z10P600', 'S50Z10P650', 'S50Z10P700', 'S50Z10P750']

optionsSymbol_2011 = ['S50M11C500', 'S50M11C550', 'S50M11C600', 'S50M11C650', 'S50M11C700', 'S50M11C750', 'S50M11C800', 'S50H11C450', 'S50H11C500', 'S50H11C550', 'S50H11C600', 'S50H11C650', 'S50H11C700', 'S50H11C750', 'S50U11C600', 'S50U11C650', 'S50U11C700', 'S50U11C750', 'S50U11C800', 'S50U11C850', 'S50Z11C550', 'S50Z11C600', 'S50Z11C650', 'S50Z11C700', 'S50Z11C750', 'S50Z11C800', 'S50Z11C850', 'S50M11P500', 'S50M11P550', 'S50M11P600', 'S50M11P650', 'S50M11P700', 'S50M11P750', 'S50M11P800', 'S50H11P450', 'S50H11P500', 'S50H11P550', 'S50H11P600', 'S50H11P650', 'S50H11P700', 'S50H11P750', 'S50U11P600', 'S50U11P650', 'S50U11P700', 'S50U11P750', 'S50U11P800', 'S50U11P850', 'S50Z11P550', 'S50Z11P600', 'S50Z11P650', 'S50Z11P700', 'S50Z11P750', 'S50Z11P800', 'S50Z11P850']
optionsSymbol_2012 = ['S50M12C550', 'S50M12C600', 'S50M12C650', 'S50M12C700', 'S50M12C750', 'S50M12C800', 'S50M12C850', 'S50M12C900', 'S50H12C550', 'S50H12C600', 'S50H12C650', 'S50H12C700', 'S50H12C750', 'S50H12C800', 'S50H12C850', 'S50H12C900', 'S50U12C550', 'S50U12C600', 'S50U12C650', 'S50U12C700', 'S50U12C750', 'S50U12C800', 'S50U12C850', 'S50U12C900', 'S50Z12C700', 'S50Z12C750', 'S50Z12C800', 'S50Z12C825', 'S50Z12C850', 'S50Z12C875', 'S50Z12C900', 'S50Z12C925', 'S50Z12C950', 'S50Z12C975', 'S50Z12C1000', 'S50M12P550', 'S50M12P600', 'S50M12P650', 'S50M12P700', 'S50M12P750', 'S50M12P800', 'S50M12P850', 'S50M12P900', 'S50H12P550', 'S50H12P600', 'S50H12P650', 'S50H12P700', 'S50H12P750', 'S50H12P800', 'S50H12P850', 'S50H12P900', 'S50U12P550', 'S50U12P600', 'S50U12P650', 'S50U12P700', 'S50U12P750', 'S50U12P800', 'S50U12P850', 'S50U12P900', 'S50Z12P700', 'S50Z12P750', 'S50Z12P800', 'S50Z12P825', 'S50Z12P850', 'S50Z12P875', 'S50Z12P900', 'S50Z12P925', 'S50Z12P950', 'S50Z12P975', 'S50Z12P1000']
optionsSymbol_2013 = ['S50M13C800', 'S50M13C850', 'S50M13C875', 'S50M13C900', 'S50M13C925', 'S50M13C950', 'S50M13C975', 'S50M13C1000', 'S50M13C1025', 'S50M13C1050', 'S50M13C1075', 'S50M13C1100', 'S50M13C1125', 'S50M13C1150', 'S50H13C750', 'S50H13C800', 'S50H13C825', 'S50H13C850', 'S50H13C875', 'S50H13C900', 'S50H13C925', 'S50H13C950', 'S50H13C975', 'S50H13C1000', 'S50H13C1025', 'S50H13C1050', 'S50H13C1075', 'S50H13C1100', 'S50U13C825', 'S50U13C850', 'S50U13C875', 'S50U13C900', 'S50U13C925', 'S50U13C950', 'S50U13C975', 'S50U13C1000', 'S50U13C1025', 'S50U13C1050', 'S50U13C1075', 'S50U13C1100', 'S50U13C1125', 'S50U13C1150', 'S50Z13C825', 'S50Z13C850', 'S50Z13C875', 'S50Z13C900', 'S50Z13C925', 'S50Z13C950', 'S50Z13C975', 'S50Z13C1000', 'S50Z13C1025', 'S50Z13C1050', 'S50Z13C1075', 'S50M13P800', 'S50M13P850', 'S50M13P875', 'S50M13P900', 'S50M13P925', 'S50M13P950', 'S50M13P975', 'S50M13P1000', 'S50M13P1025', 'S50M13P1050', 'S50M13P1075', 'S50M13P1100', 'S50M13P1125', 'S50M13P1150', 'S50H13P750', 'S50H13P800', 'S50H13P825', 'S50H13P850', 'S50H13P875', 'S50H13P900', 'S50H13P925', 'S50H13P950', 'S50H13P975', 'S50H13P1000', 'S50H13P1025', 'S50H13P1050', 'S50H13P1075', 'S50H13P1100', 'S50U13P825', 'S50U13P850', 'S50U13P875', 'S50U13P900', 'S50U13P925', 'S50U13P950', 'S50U13P975', 'S50U13P1000', 'S50U13P1025', 'S50U13P1050', 'S50U13P1075', 'S50U13P1100', 'S50U13P1125', 'S50U13P1150', 'S50Z13P825', 'S50Z13P850', 'S50Z13P875', 'S50Z13P900', 'S50Z13P925', 'S50Z13P950', 'S50Z13P975', 'S50Z13P1000', 'S50Z13P1025', 'S50Z13P1050', 'S50Z13P1075']
optionsSymbol_2014 = ['S50M14C775', 'S50M14C800', 'S50M14C825', 'S50M14C850', 'S50M14C875', 'S50M14C900', 'S50M14C925', 'S50M14C950', 'S50M14C975', 'S50M14C1000', 'S50M14C1025', 'S50M14C1050', 'S50H14C775', 'S50H14C800', 'S50H14C825', 'S50H14C850', 'S50H14C875', 'S50H14C900', 'S50H14C925', 'S50H14C950', 'S50H14C975', 'S50H14C1000', 'S50H14C1025', 'S50H14C1050', 'S50H14C1075', 'S50U14C875', 'S50U14C900', 'S50U14C925', 'S50U14C950', 'S50U14C975', 'S50U14C1000', 'S50U14C1025', 'S50U14C1050', 'S50U14C1075', 'S50U14C1100', 'S50U14C1125', 'S50Z14C925', 'S50Z14C950', 'S50Z14C975', 'S50Z14C1000', 'S50Z14C1025', 'S50Z14C1050', 'S50Z14C1075', 'S50Z14C1100', 'S50Z14C1125', 'S50M14P775', 'S50M14P800', 'S50M14P825', 'S50M14P850', 'S50M14P875', 'S50M14P900', 'S50M14P925', 'S50M14P950', 'S50M14P975', 'S50M14P1000', 'S50M14P1025', 'S50M14P1050', 'S50H14P775', 'S50H14P800', 'S50H14P825', 'S50H14P850', 'S50H14P875', 'S50H14P900', 'S50H14P925', 'S50H14P950', 'S50H14P975', 'S50H14P1000', 'S50H14P1025', 'S50H14P1050', 'S50H14P1075', 'S50U14P875', 'S50U14P900', 'S50U14P925', 'S50U14P950', 'S50U14P975', 'S50U14P1000', 'S50U14P1025', 'S50U14P1050', 'S50U14P1075', 'S50U14P1100', 'S50U14P1125', 'S50Z14P925', 'S50Z14P950', 'S50Z14P975', 'S50Z14P1000', 'S50Z14P1025', 'S50Z14P1050', 'S50Z14P1075', 'S50Z14P1100', 'S50Z14P1125']
optionsSymbol_2015 = ['S50M15C925', 'S50M15C950', 'S50M15C975', 'S50M15C1000', 'S50M15C1025', 'S50M15C1050', 'S50M15C1075', 'S50M15C1100', 'S50M15C1125', 'S50H15C925', 'S50H15C950', 'S50H15C975', 'S50H15C1000', 'S50H15C1025', 'S50H15C1050', 'S50H15C1075', 'S50H15C1100', 'S50H15C1125', 'S50U15C800', 'S50U15C825', 'S50U15C850', 'S50U15C875', 'S50U15C900', 'S50U15C925', 'S50U15C950', 'S50U15C975', 'S50U15C1000', 'S50U15C1025', 'S50U15C1050', 'S50U15C1075', 'S50U15C1100', 'S50Z15C750', 'S50Z15C775', 'S50Z15C800', 'S50Z15C825', 'S50Z15C850', 'S50Z15C875', 'S50Z15C900', 'S50Z15C925', 'S50Z15C950', 'S50Z15C975', 'S50Z15C1000', 'S50Z15C1025', 'S50Z15C1050', 'S50M15P925', 'S50M15P950', 'S50M15P975', 'S50M15P1000', 'S50M15P1025', 'S50M15P1050', 'S50M15P1075', 'S50M15P1100', 'S50M15P1125', 'S50H15P925', 'S50H15P950', 'S50H15P975', 'S50H15P1000', 'S50H15P1025', 'S50H15P1050', 'S50H15P1075', 'S50H15P1100', 'S50H15P1125', 'S50U15P800', 'S50U15P825', 'S50U15P850', 'S50U15P875', 'S50U15P900', 'S50U15P925', 'S50U15P950', 'S50U15P975', 'S50U15P1000', 'S50U15P1025', 'S50U15P1050', 'S50U15P1075', 'S50U15P1100', 'S50Z15P750', 'S50Z15P775', 'S50Z15P800', 'S50Z15P825', 'S50Z15P850', 'S50Z15P875', 'S50Z15P900', 'S50Z15P925', 'S50Z15P950', 'S50Z15P975', 'S50Z15P1000', 'S50Z15P1025', 'S50Z15P1050']

optionsSymbol_2016 = ['S50M16C700', 'S50M16C725', 'S50M16C750', 'S50M16C775', 'S50M16C800', 'S50M16C825', 'S50M16C850', 'S50M16C875', 'S50M16C900', 'S50M16C925', 'S50M16C950', 'S50M16C975', 'S50H16C700', 'S50H16C725', 'S50H16C750', 'S50H16C775', 'S50H16C800', 'S50H16C825', 'S50H16C850', 'S50H16C875', 'S50H16C900', 'S50H16C925', 'S50H16C950', 'S50H16C975', 'S50U16C800', 'S50U16C825', 'S50U16C850', 'S50U16C875', 'S50U16C900', 'S50U16C925', 'S50U16C950', 'S50U16C975', 'S50U16C1000', 'S50U16C1025', 'S50U16C1050', 'S50Z16C850', 'S50Z16C875', 'S50Z16C900', 'S50Z16C925', 'S50Z16C950', 'S50Z16C975', 'S50Z16C1000', 'S50Z16C1025', 'S50Z16C1050', 'S50M16P700', 'S50M16P725', 'S50M16P750', 'S50M16P775', 'S50M16P800', 'S50M16P825', 'S50M16P850', 'S50M16P875', 'S50M16P900', 'S50M16P925', 'S50M16P950', 'S50M16P975', 'S50H16P700', 'S50H16P725', 'S50H16P750', 'S50H16P775', 'S50H16P800', 'S50H16P825', 'S50H16P850', 'S50H16P875', 'S50H16P900', 'S50H16P925', 'S50H16P950', 'S50H16P975', 'S50U16P800', 'S50U16P825', 'S50U16P850', 'S50U16P875', 'S50U16P900', 'S50U16P925', 'S50U16P950', 'S50U16P975', 'S50U16P1000', 'S50U16P1025', 'S50U16P1050', 'S50Z16P850', 'S50Z16P875', 'S50Z16P900', 'S50Z16P925', 'S50Z16P950', 'S50Z16P975', 'S50Z16P1000', 'S50Z16P1025', 'S50Z16P1050']
optionsSymbol_2017 = ['S50M17C900', 'S50M17C925', 'S50M17C950', 'S50M17C975', 'S50M17C1000', 'S50M17C1025', 'S50M17C1050', 'S50H17C850', 'S50H17C875', 'S50H17C900', 'S50H17C925', 'S50H17C950', 'S50H17C975', 'S50H17C1000', 'S50H17C1025', 'S50H17C1050', 'S50U17C925', 'S50U17C950', 'S50U17C975', 'S50U17C1000', 'S50U17C1025', 'S50U17C1050', 'S50U17C1075', 'S50U17C1100', 'S50U17C1125', 'S50Z17C950', 'S50Z17C975', 'S50Z17C1000', 'S50Z17C1025', 'S50Z17C1050', 'S50Z17C1075', 'S50Z17C1100', 'S50Z17C1125', 'S50Z17C1150', 'S50Z17C1175', 'S50M17P900', 'S50M17P925', 'S50M17P950', 'S50M17P975', 'S50M17P1000', 'S50M17P1025', 'S50M17P1050', 'S50H17P850', 'S50H17P875', 'S50H17P900', 'S50H17P925', 'S50H17P950', 'S50H17P975', 'S50H17P1000', 'S50H17P1025', 'S50H17P1050', 'S50U17P925', 'S50U17P950', 'S50U17P975', 'S50U17P1000', 'S50U17P1025', 'S50U17P1050', 'S50U17P1075', 'S50U17P1100', 'S50U17P1125', 'S50Z17P950', 'S50Z17P975', 'S50Z17P1000', 'S50Z17P1025', 'S50Z17P1050', 'S50Z17P1075', 'S50Z17P1100', 'S50Z17P1125', 'S50Z17P1150', 'S50Z17P1175']
optionsSymbol_2018 = ['S50M18C1000', 'S50M18C1025', 'S50M18C1050', 'S50M18C1075', 'S50M18C1100', 'S50M18C1125', 'S50M18C1150', 'S50M18C1175', 'S50M18C1200', 'S50M18C1225', 'S50M18C1250', 'S50H18C1025', 'S50H18C1050', 'S50H18C1075', 'S50H18C1100', 'S50H18C1125', 'S50H18C1150', 'S50H18C1175', 'S50H18C1200', 'S50H18C1225', 'S50H18C1250', 'S50U18C1000', 'S50U18C1025', 'S50U18C1050', 'S50U18C1075', 'S50U18C1100', 'S50U18C1125', 'S50U18C1150', 'S50U18C1175', 'S50U18C1200', 'S50U18C1225', 'S50U18C1250', 'S50Z18C1000', 'S50Z18C1025', 'S50Z18C1050', 'S50Z18C1075', 'S50Z18C1100', 'S50Z18C1125', 'S50Z18C1150', 'S50Z18C1175', 'S50Z18C1200', 'S50M18P1000', 'S50M18P1025', 'S50M18P1050', 'S50M18P1075', 'S50M18P1100', 'S50M18P1125', 'S50M18P1150', 'S50M18P1175', 'S50M18P1200', 'S50M18P1225', 'S50M18P1250', 'S50H18P1025', 'S50H18P1050', 'S50H18P1075', 'S50H18P1100', 'S50H18P1125', 'S50H18P1150', 'S50H18P1175', 'S50H18P1200', 'S50H18P1225', 'S50H18P1250', 'S50U18P1000', 'S50U18P1025', 'S50U18P1050', 'S50U18P1075', 'S50U18P1100', 'S50U18P1125', 'S50U18P1150', 'S50U18P1175', 'S50U18P1200', 'S50U18P1225', 'S50U18P1250', 'S50Z18P1000', 'S50Z18P1025', 'S50Z18P1050', 'S50Z18P1075', 'S50Z18P1100', 'S50Z18P1125', 'S50Z18P1150', 'S50Z18P1175', 'S50Z18P1200']
optionsSymbol_2019 = ['S50M19C975', 'S50M19C1000', 'S50M19C1025', 'S50M19C1050', 'S50M19C1075', 'S50M19C1100', 'S50M19C1125', 'S50M19C1150', 'S50M19C1175', 'S50M19C1200', 'S50H19C975', 'S50H19C1000', 'S50H19C1025', 'S50H19C1050', 'S50H19C1075', 'S50H19C1100', 'S50H19C1125', 'S50H19C1150', 'S50H19C1175', 'S50H19C1200', 'S50U19C1000', 'S50U19C1025', 'S50U19C1050', 'S50U19C1075', 'S50U19C1100', 'S50U19C1125', 'S50U19C1150', 'S50U19C1175', 'S50U19C1200', 'S50Z19C1000', 'S50Z19C1025', 'S50Z19C1050', 'S50Z19C1075', 'S50Z19C1100', 'S50Z19C1125', 'S50Z19C1150', 'S50Z19C1175', 'S50Z19C1200', 'S50M19P975', 'S50M19P1000', 'S50M19P1025', 'S50M19P1050', 'S50M19P1075', 'S50M19P1100', 'S50M19P1125', 'S50M19P1150', 'S50M19P1175', 'S50M19P1200', 'S50H19P975', 'S50H19P1000', 'S50H19P1025', 'S50H19P1050', 'S50H19P1075', 'S50H19P1100', 'S50H19P1125', 'S50H19P1150', 'S50H19P1175', 'S50H19P1200', 'S50U19P1000', 'S50U19P1025', 'S50U19P1050', 'S50U19P1075', 'S50U19P1100', 'S50U19P1125', 'S50U19P1150', 'S50U19P1175', 'S50U19P1200', 'S50Z19P1000', 'S50Z19P1025', 'S50Z19P1050', 'S50Z19P1075', 'S50Z19P1100', 'S50Z19P1125', 'S50Z19P1150', 'S50Z19P1175', 'S50Z19P1200']
optionsSymbol_2020 = ['S50M20C575', 'S50M20C600', 'S50M20C625', 'S50M20C650', 'S50M20C675', 'S50M20C700', 'S50M20C725', 'S50M20C750', 'S50M20C775', 'S50M20C800', 'S50M20C825', 'S50M20C850', 'S50M20C875', 'S50M20C900', 'S50M20C925', 'S50M20C950', 'S50M20C975', 'S50M20C1000', 'S50M20C1025', 'S50M20C1050', 'S50M20C1075', 'S50M20C1100', 'S50M20C1125', 'S50H20C575', 'S50H20C600', 'S50H20C625', 'S50H20C650', 'S50H20C675', 'S50H20C700', 'S50H20C725', 'S50H20C750', 'S50H20C775', 'S50H20C800', 'S50H20C825', 'S50H20C850', 'S50H20C875', 'S50H20C900', 'S50H20C925', 'S50H20C950', 'S50H20C975', 'S50H20C1000', 'S50H20C1025', 'S50H20C1050', 'S50H20C1075', 'S50H20C1100', 'S50H20C1125', 'S50H20C1150', 'S50U20C625', 'S50U20C650', 'S50U20C675', 'S50U20C700', 'S50U20C725', 'S50U20C750', 'S50U20C775', 'S50U20C800', 'S50U20C825', 'S50U20C850', 'S50U20C875', 'S50U20C900', 'S50U20C925', 'S50U20C950', 'S50U20C975', 'S50U20C1000', 'S50U20C1025', 'S50U20C1050', 'S50U20C1075', 'S50Z20C650', 'S50Z20C675', 'S50Z20C700', 'S50Z20C725', 'S50Z20C750', 'S50Z20C775', 'S50Z20C800', 'S50Z20C825', 'S50Z20C850', 'S50Z20C875', 'S50Z20C900', 'S50Z20C925', 'S50Z20C950', 'S50Z20C975', 'S50Z20C1000', 'S50Z20C1025', 'S50Z20C1050', 'S50M20P575', 'S50M20P600', 'S50M20P625', 'S50M20P650', 'S50M20P675', 'S50M20P700', 'S50M20P725', 'S50M20P750', 'S50M20P775', 'S50M20P800', 'S50M20P825', 'S50M20P850', 'S50M20P875', 'S50M20P900', 'S50M20P925', 'S50M20P950', 'S50M20P975', 'S50M20P1000', 'S50M20P1025', 'S50M20P1050', 'S50M20P1075', 'S50M20P1100', 'S50M20P1125', 'S50H20P575', 'S50H20P600', 'S50H20P625', 'S50H20P650', 'S50H20P675', 'S50H20P700', 'S50H20P725', 'S50H20P750', 'S50H20P775', 'S50H20P800', 'S50H20P825', 'S50H20P850', 'S50H20P875', 'S50H20P900', 'S50H20P925', 'S50H20P950', 'S50H20P975', 'S50H20P1000', 'S50H20P1025', 'S50H20P1050', 'S50H20P1075', 'S50H20P1100', 'S50H20P1125', 'S50H20P1150', 'S50U20P625', 'S50U20P650', 'S50U20P675', 'S50U20P700', 'S50U20P725', 'S50U20P750', 'S50U20P775', 'S50U20P800', 'S50U20P825', 'S50U20P850', 'S50U20P875', 'S50U20P900', 'S50U20P925', 'S50U20P950', 'S50U20P975', 'S50U20P1000', 'S50U20P1025', 'S50U20P1050', 'S50U20P1075', 'S50Z20P650', 'S50Z20P675', 'S50Z20P700', 'S50Z20P725', 'S50Z20P750', 'S50Z20P775', 'S50Z20P800', 'S50Z20P825', 'S50Z20P850', 'S50Z20P875', 'S50Z20P900', 'S50Z20P925', 'S50Z20P950', 'S50Z20P975', 'S50Z20P1000', 'S50Z20P1025', 'S50Z20P1050']

optionsSymbol_2021 = ['S50M21C800', 'S50M21C825', 'S50M21C850', 'S50M21C875', 'S50M21C900', 'S50M21C925', 'S50M21C950', 'S50M21C975', 'S50M21C1000', 'S50M21C1025', 'S50M21C1050', 'S50M21C1075', 'S50M21C1100', 'S50H21C650', 'S50H21C675', 'S50H21C700', 'S50H21C725', 'S50H21C750', 'S50H21C775', 'S50H21C800', 'S50H21C825', 'S50H21C850', 'S50H21C875', 'S50H21C900', 'S50H21C925', 'S50H21C950', 'S50H21C975', 'S50H21C1000', 'S50H21C1025', 'S50H21C1050', 'S50H21C1075', 'S50U21C800', 'S50U21C825', 'S50U21C850', 'S50U21C875', 'S50U21C900', 'S50U21C925', 'S50U21C950', 'S50U21C975', 'S50U21C1000', 'S50U21C1025', 'S50U21C1050', 'S50U21C1075', 'S50U21C1100', 'S50Z21C800', 'S50Z21C825', 'S50Z21C850', 'S50Z21C875', 'S50Z21C900', 'S50Z21C925', 'S50Z21C950', 'S50Z21C975', 'S50Z21C1000', 'S50Z21C1025', 'S50Z21C1050', 'S50Z21C1075', 'S50Z21C1100', 'S50M21P800', 'S50M21P825', 'S50M21P850', 'S50M21P875', 'S50M21P900', 'S50M21P925', 'S50M21P950', 'S50M21P975', 'S50M21P1000', 'S50M21P1025', 'S50M21P1050', 'S50M21P1075', 'S50M21P1100', 'S50H21P650', 'S50H21P675', 'S50H21P700', 'S50H21P725', 'S50H21P750', 'S50H21P775', 'S50H21P800', 'S50H21P825', 'S50H21P850', 'S50H21P875', 'S50H21P900', 'S50H21P925', 'S50H21P950', 'S50H21P975', 'S50H21P1000', 'S50H21P1025', 'S50H21P1050', 'S50H21P1075', 'S50U21P800', 'S50U21P825', 'S50U21P850', 'S50U21P875', 'S50U21P900', 'S50U21P925', 'S50U21P950', 'S50U21P975', 'S50U21P1000', 'S50U21P1025', 'S50U21P1050', 'S50U21P1075', 'S50U21P1100', 'S50Z21P800', 'S50Z21P825', 'S50Z21P850', 'S50Z21P875', 'S50Z21P900', 'S50Z21P925', 'S50Z21P950', 'S50Z21P975', 'S50Z21P1000', 'S50Z21P1025', 'S50Z21P1050', 'S50Z21P1075', 'S50Z21P1100']
optionsSymbol_2022 = ['S50M22C850', 'S50M22C875', 'S50M22C900', 'S50M22C925', 'S50M22C950', 'S50M22C975', 'S50M22C1000', 'S50M22C1025', 'S50M22C1050', 'S50M22C1075', 'S50M22C1100', 'S50M22C1125', 'S50H22C825', 'S50H22C850', 'S50H22C875', 'S50H22C900', 'S50H22C925', 'S50H22C950', 'S50H22C975', 'S50H22C1000', 'S50H22C1025', 'S50H22C1050', 'S50H22C1075', 'S50H22C1100', 'S50H22C1125', 'S50U22C850', 'S50U22C875', 'S50U22C900', 'S50U22C925', 'S50U22C950', 'S50U22C975', 'S50U22C1000', 'S50U22C1025', 'S50U22C1050', 'S50U22C1075', 'S50U22C1100', 'S50U22C1125', 'S50Z22C825', 'S50Z22C850', 'S50Z22C875', 'S50Z22C900', 'S50Z22C925', 'S50Z22C950', 'S50Z22C975', 'S50Z22C1000', 'S50Z22C1025', 'S50Z22C1050', 'S50Z22C1075', 'S50Z22C1100', 'S50M22P850', 'S50M22P875', 'S50M22P900', 'S50M22P925', 'S50M22P950', 'S50M22P975', 'S50M22P1000', 'S50M22P1025', 'S50M22P1050', 'S50M22P1075', 'S50M22P1100', 'S50M22P1125', 'S50H22P825', 'S50H22P850', 'S50H22P875', 'S50H22P900', 'S50H22P925', 'S50H22P950', 'S50H22P975', 'S50H22P1000', 'S50H22P1025', 'S50H22P1050', 'S50H22P1075', 'S50H22P1100', 'S50H22P1125', 'S50U22P850', 'S50U22P875', 'S50U22P900', 'S50U22P925', 'S50U22P950', 'S50U22P975', 'S50U22P1000', 'S50U22P1025', 'S50U22P1050', 'S50U22P1075', 'S50U22P1100', 'S50U22P1125', 'S50Z22P825', 'S50Z22P850', 'S50Z22P875', 'S50Z22P900', 'S50Z22P925', 'S50Z22P950', 'S50Z22P975', 'S50Z22P1000', 'S50Z22P1025', 'S50Z22P1050', 'S50Z22P1075', 'S50Z22P1100']
optionsSymbol_2023 = ['S50M23C800', 'S50M23C825', 'S50M23C850', 'S50M23C875', 'S50M23C900', 'S50M23C925', 'S50M23C950', 'S50M23C975', 'S50M23C1000', 'S50M23C1025', 'S50M23C1050', 'S50M23C1075', 'S50M23C1100', 'S50M23C1125', 'S50H23C800', 'S50H23C825', 'S50H23C850', 'S50H23C875', 'S50H23C900', 'S50H23C925', 'S50H23C950', 'S50H23C975', 'S50H23C1000', 'S50H23C1025', 'S50H23C1050', 'S50H23C1075', 'S50H23C1100', 'S50H23C1125', 'S50U23C800', 'S50U23C825', 'S50U23C850', 'S50U23C875', 'S50U23C900', 'S50U23C925', 'S50U23C950', 'S50U23C975', 'S50U23C1000', 'S50U23C1025', 'S50U23C1050', 'S50U23C1075', 'S50Z23C750', 'S50Z23C775', 'S50Z23C800', 'S50Z23C825', 'S50Z23C850', 'S50Z23C875', 'S50Z23C900', 'S50Z23C925', 'S50Z23C950', 'S50Z23C975', 'S50Z23C1000', 'S50Z23C1025', 'S50Z23C1050', 'S50Z23C1075', 'S50M23P800', 'S50M23P825', 'S50M23P850', 'S50M23P875', 'S50M23P900', 'S50M23P925', 'S50M23P950', 'S50M23P975', 'S50M23P1000', 'S50M23P1025', 'S50M23P1050', 'S50M23P1075', 'S50M23P1100', 'S50M23P1125', 'S50H23P800', 'S50H23P825', 'S50H23P850', 'S50H23P875', 'S50H23P900', 'S50H23P925', 'S50H23P950', 'S50H23P975', 'S50H23P1000', 'S50H23P1025', 'S50H23P1050', 'S50H23P1075', 'S50H23P1100', 'S50H23P1125', 'S50U23P800', 'S50U23P825', 'S50U23P850', 'S50U23P875', 'S50U23P900', 'S50U23P925', 'S50U23P950', 'S50U23P975', 'S50U23P1000', 'S50U23P1025', 'S50U23P1050', 'S50U23P1075', 'S50Z23P750', 'S50Z23P775', 'S50Z23P800', 'S50Z23P825', 'S50Z23P850', 'S50Z23P875', 'S50Z23P900', 'S50Z23P925', 'S50Z23P950', 'S50Z23P975', 'S50Z23P1000', 'S50Z23P1025', 'S50Z23P1050', 'S50Z23P1075']

##### **Export to .csv**

In [ ]:
get_options_data(optionsSymbol_2007).to_csv('optionsData_2007.csv', index=False)
get_options_data(optionsSymbol_2008).to_csv('optionsData_2008.csv', index=False)
get_options_data(optionsSymbol_2009).to_csv('optionsData_2009.csv', index=False)
get_options_data(optionsSymbol_2010).to_csv('optionsData_2010.csv', index=False)

get_options_data(optionsSymbol_2011).to_csv('optionsData_2011.csv', index=False)
get_options_data(optionsSymbol_2012).to_csv('optionsData_2012.csv', index=False)
get_options_data(optionsSymbol_2013).to_csv('optionsData_2013.csv', index=False)
get_options_data(optionsSymbol_2014).to_csv('optionsData_2014.csv', index=False)
get_options_data(optionsSymbol_2015).to_csv('optionsData_2015.csv', index=False)

get_options_data(optionsSymbol_2016).to_csv('optionsData_2016.csv', index=False)
get_options_data(optionsSymbol_2017).to_csv('optionsData_2017.csv', index=False)
get_options_data(optionsSymbol_2018).to_csv('optionsData_2018.csv', index=False)
get_options_data(optionsSymbol_2019).to_csv('optionsData_2019.csv', index=False)
get_options_data(optionsSymbol_2020).to_csv('optionsData_2020.csv', index=False)

get_options_data(optionsSymbol_2020).to_csv('optionsData_2021.csv', index=False)
get_options_data(optionsSymbol_2020).to_csv('optionsData_2022.csv', index=False)
get_options_data(optionsSymbol_2020).to_csv('optionsData_2023.csv', index=False)